In [68]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters = 1000
# eval_interval = 2500
learning_rate = 3e-4
eval_iters = 250

cpu


In [3]:
# Managing and Opening Text File

with open('wizard_of_oz.txt','r',encoding='utf-8') as f:
    text = f.read()

print(len(text))
    

207797


In [66]:
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '&', '(', ')', ',', '-', '.', '0', '1', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '—', '‘', '’', '“', '”', '\ufeff']


In [19]:
# Tokenizer has an encoder - convert each element of chars to integer
# , decoder - translates and convert these integers into str

string_to_int = {ch:i for i,ch in enumerate(chars)}
int_to_string = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])


In [24]:
# # Have this as a long sequence of integers
# data = torch.tensor(encode(text),dtype = torch.long)
# print(data[:100])

tensor([72, 34, 48, 45,  1, 37, 55, 54, 44, 45, 58, 46, 61, 52,  1, 37, 49, 66,
        41, 58, 44,  1, 55, 46,  1, 29, 66,  0,  0, 42, 65,  1, 26,  8,  1, 20,
        58, 41, 54, 51,  1, 16, 41, 61, 53,  0,  0,  0, 34, 48, 49, 59,  1, 42,
        55, 55, 51,  1, 49, 59,  1, 44, 45, 44, 49, 43, 41, 60, 45, 44,  1, 60,
        55,  1, 53, 65,  1, 47, 55, 55, 44,  1, 46, 58, 49, 45, 54, 44,  1,  3,
         1, 43, 55, 53, 58, 41, 44, 45,  0, 27])


In [69]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs:')
# print(x.shape)
print(x)
print('targets:')
print(y)

inputs:
tensor([[ 1, 41, 54, 44,  1, 48, 45,  1],
        [59, 60, 14, 71,  0,  0, 70, 23],
        [45,  1, 42, 52, 55, 55, 44,  1],
        [45, 54,  6,  0, 59, 48, 55, 63]])
targets:
tensor([[41, 54, 44,  1, 48, 45,  1, 48],
        [60, 14, 71,  0,  0, 70, 23,  1],
        [ 1, 42, 52, 55, 55, 44,  1, 49],
        [54,  6,  0, 59, 48, 55, 63, 49]])


In [71]:
# block_size = 8

# x = train_data[:block_size]
# y = train_data[1:block_size+1]

# for t in range(block_size):
#     context = x[:t+1]
#     target = y[t]
#     print(f"When input is {context} target is {target}")

In [65]:
# When nn.Module function is given as input it acts as a learnable parameter
class BigramLanguageModel(nn.Module):
    def __init__(self,vocab_size):
        super().__init__()
        # Below is a learnable parameter and a lookup table, giant grid of what the predictions can look like
        self.token_embeddings_table = nn.Embedding(vocab_size,vocab_size)
    
    def forward(selfmindex,targets):
        logits = self.token_embeddings_table(index)
        # Batch and time are not as important so they can be belended together, channels contains the vocab size
        batch,time,channels = logits.shape
        # view - changes the shape, we rehsape because we want the pytorch input to be in the expected shape
        logits = logits.view(batch*time,channels)
        targets = targets.view(batch*time)
        loss = F.cross_entropy(logits,targets)
        return logits